<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Evolution CA"

**Tags:** #fec #finance #snippet #operation #ca #revenue

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser l'évolution du CA entre l'année N et l'année N-1.

## Input

### Import libraries

In [6]:
import naas_data_product

### Setup Variables

In [11]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin"
openai_api_key = naas.secret.get(name="OPENAI_API_KEY") or "ENTER_YOUR_OPENAI_API_KEY"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_evolution_ca"
prompt_data = """
Tu es un analyste comptable et financier.
Je vais te passer un jeu de données sur l'évolution du CA d'une entreprise entre une ou plusieurs années avec les valeurs cumulées (VALUE_CUM) et décumulées (VALUE) par mois.
Peux-tu me donner un bref aperçu factuel de l'évolution du CA et identifier les fortes variations positives et négatives?
Peux-tu commencer ton analyse par "Le CA de l'entreprise a ..." et afficher les chiffres au format suivant: 26 052 €?
"""

## Model

### Récupération du dernier fichier input

In [8]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/bdd_fin/20230605122348_FEC_bdd_fin.csv
✅ Row fetched: 9064


,NOM_FICHIER,ENTITY,PERIOD,DATE,MONTH,MONTH_INDEX,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT,VALUE
0,000000001FEC20221231.txt,000000001,202212,2018-01-01,Jan,1,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.6,0.0,4006.6


### Création du dataset "EVOLUTION_CA"

In [9]:
def prep_data(df_init, ):
    # Init
    df = df_init.copy()
    df_output = pd.DataFrame()
    
    # Filtre COMPTE_NUM = Chiffre d'Affaire (RUBRIQUE N1)
    df = df[df["COMPTE_NUM"].str.contains(r"^70|^71|^72")]

    # Regroupement
    to_group = [
        "ENTITY",
        "PERIOD",
        "DATE",
    ]
    to_agg = {"VALUE": "sum"}
    df = df.groupby(to_group, as_index=False).agg(to_agg).sort_values(by=["ENTITY", "PERIOD", "DATE"], ascending=[True, False, True])
    
    # Format columns
    df["DATE"] = pd.to_datetime(df["DATE"])
    df["VALUE"] = df["VALUE"].abs()
    
    # Reindex value
    filters = df[["ENTITY", "PERIOD"]].drop_duplicates(ignore_index=True)
    for index, row in filters.iterrows():
        entity = row["ENTITY"]
        period = row["PERIOD"]
        tmp_df = df[(df["ENTITY"] == entity) & (df["PERIOD"] == period)]
        
        # Create date range
        d_start = tmp_df.loc[tmp_df.index[0], "DATE"]
        start_year = d_start.year
        start_month = d_start.month
        start = f'{start_year}-{start_month}-01'
        d_end = tmp_df.loc[tmp_df.index[-1], "DATE"]
        end = d_end
        idx = pd.date_range(start, end, freq="D")
        
        # Reindex by DATE
        tmp_df.set_index("DATE", drop=True, inplace=True)
        tmp_df.index = pd.DatetimeIndex(tmp_df.index)
        tmp_df = tmp_df.reindex(idx, fill_value=0)
        
        # Enforce data
        tmp_df["ENTITY"] = entity
        tmp_df["PERIOD"] = period
        tmp_df.insert(loc=2, column="DATE", value=pd.DatetimeIndex(tmp_df.index))
        
        # Groupby analytics
        tmp_df["DATE"] = tmp_df["DATE"].dt.strftime("%Y-%m")
        tmp_df = tmp_df.groupby(to_group, as_index=False).agg(to_agg)
        tmp_df["VALUE_CUM"] = tmp_df["VALUE"].cumsum()
        
        # Concat
        df_output = pd.concat([df_output, tmp_df])
    return df_output.reset_index(drop=True)

data_prep = prep_data(df_input)
print("Row fetched:", len(data_prep))
data_prep.head(12)

Row fetched: 48


,ENTITY,PERIOD,DATE,VALUE,VALUE_CUM
0,000000000,201812,2018-01,41602.29,41602.29
1,000000000,201812,2018-02,37467.92,79070.21
2,000000000,201812,2018-03,51952.85,131023.06
3,000000000,201812,2018-04,48923.29,179946.35
4,000000000,201812,2018-05,55113.09,235059.44
5,000000000,201812,2018-06,67334.60,302394.04
6,000000000,201812,2018-07,63284.60,365678.64
7,000000000,201812,2018-08,0.00,365678.64
8,000000000,201812,2018-09,49574.30,415252.94
9,000000000,201812,2018-10,51820.05,467072.99


### Création des analyses N et N-1 par période

In [12]:
def create_data_ca(df_init):
    # Init
    df = df_init.copy()
    df_output = pd.DataFrame()
    
    # Get periods by entity
    entities = df[["ENTITY"]].drop_duplicates(ignore_index=True)
    for index, row in entities.iterrows():
        entity = row["ENTITY"]
        tmp_df = df.copy()
        tmp_df = tmp_df[(tmp_df["ENTITY"] == entity)]
        
        # Get periods
        periods = tmp_df[["PERIOD"]].drop_duplicates(ignore_index=True)
        
        for i, r in periods.iterrows():
            period = periods["PERIOD"][i]
            if i < len(periods) - 1:
                period_comp = periods["PERIOD"][i+1]
            else:
                period_comp = None
            print(f"➡️ Periode: {period_comp} -> {period}")
            tmp_df2 = tmp_df[(tmp_df["PERIOD"].isin([period, period_comp]))].reset_index(drop=True)
            print("Row fetched:", len(tmp_df2))
            tmp_df2.loc[tmp_df2["PERIOD"] == period, "GROUP"] = "N"
            tmp_df2.loc[tmp_df2["PERIOD"] == period_comp, "GROUP"] = "N-1"
            tmp_df2["PERIOD"] = period
            analysis = get_ia_analysis(openai_api_key, tmp_df2, prompt_data)
            tmp_df2["AI_ANALYSIS"] = analysis
            
            # Concat
            df_output = pd.concat([df_output, tmp_df2])
            
    # Column to be displayed
    df_output["MONTH_INDEX"] = pd.to_datetime(df_output["DATE"], format="%Y-%m").dt.strftime("%m")
    df_output["VALUE_D"] = (df_output["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    df_output["VALUE_CUM_D"] = (df_output["VALUE_CUM"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    return df_output.reset_index(drop=True)

df_evolution_ca = create_data_ca(data_prep)
print("Row fetched:", len(df_evolution_ca))
df_evolution_ca.head(24)

➡️ Periode: 201712 -> 201812
Row fetched: 24
🤖 IA: Le CA de l'entreprise a augmenté de 2,5% entre 2017 et 2018, passant de 549 403 € à 551 927 €. Les mois avec les plus fortes variations positives sont mars 2018 (+15,5%), juin 2018 (+16,2%) et septembre 2018 (+14,5%). Les mois avec les plus fortes variations négatives sont février 2018 (-10,0%), avril 2018 (-6,0%) et novembre 2018 (-10,5%).
➡️ Periode: None -> 201712
Row fetched: 12
🤖 IA: Le CA de l'entreprise a augmenté de manière régulière tout au long de l'année 2017, passant de 42 550 € en janvier à 54 940 € en septembre, avant de connaître une forte hausse en octobre avec un chiffre d'affaires de 55 081 €. Le mois de novembre a connu une baisse significative avec un CA de 44 492 €, mais l'année s'est terminée sur une note positive avec un chiffre d'affaires de 50 667 € en décembre. Les variations les plus importantes ont été enregistrées en mars avec une augmentation de 23 916 € et en octobre avec une augmentation de 6 779 €. La p

,ENTITY,PERIOD,DATE,VALUE,VALUE_CUM,GROUP,AI_ANALYSIS,MONTH_INDEX,VALUE_D,VALUE_CUM_D
0,000000000,201812,2018-01,41602.29,41602.29,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",01,41.6 k€,41.6 k€
1,000000000,201812,2018-02,37467.92,79070.21,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",02,37.5 k€,79.1 k€
2,000000000,201812,2018-03,51952.85,131023.06,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",03,52.0 k€,131.0 k€
3,000000000,201812,2018-04,48923.29,179946.35,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",04,48.9 k€,179.9 k€
4,000000000,201812,2018-05,55113.09,235059.44,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",05,55.1 k€,235.1 k€
5,000000000,201812,2018-06,67334.60,302394.04,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",06,67.3 k€,302.4 k€
6,000000000,201812,2018-07,63284.60,365678.64,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",07,63.3 k€,365.7 k€
7,000000000,201812,2018-08,0.00,365678.64,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",08,0.0 k€,365.7 k€
8,000000000,201812,2018-09,49574.30,415252.94,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",09,49.6 k€,415.3 k€
9,000000000,201812,2018-10,51820.05,467072.99,N,"Le CA de l'entreprise a augmenté de 2,5% entre...",10,51.8 k€,467.1 k€


## Output

### Sauvegarde des fichiers en csv

In [13]:
save_df(df_evolution_ca, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/dataset_evolution_ca/20230606121639_FEC_dataset_evolution_ca.csv
